In [57]:
from transformers import RobertaTokenizerFast, AutoModelForSequenceClassification
import numpy as np 
import pandas as pd
import torch

In [58]:
tokenizer = RobertaTokenizerFast.from_pretrained("./Commonlit-RoBERTa-Base/tokenizer")

file ./Commonlit-RoBERTa-Base/tokenizer\config.json not found


In [59]:
model = AutoModelForSequenceClassification.from_pretrained("./Commonlit-RoBERTa-Base")

In [66]:
gutenberg_df = pd.read_csv('./Gutenberg-Excerpts/gutenberg-excerpts.csv')
gutenberg_df

,book,author,url,excerpt
0,Pride and Prejudice,Jane Austen,https://www.gutenberg.org/files/1342/1342-h/13...,"When Jane and Elizabeth were alone, the former..."
1,Alice's Adventures in Wonderland,Lewis Carroll,https://www.gutenberg.org/files/11/11-h/11-h.htm,“Mine is a long and a sad tale!” said the Mous...
2,"Frankenstein; Or, The Modern Prometheus",Mary Wollstonecraft Shelley,https://www.gutenberg.org/files/84/84-h/84-h.htm,"When my father returned from Milan, he found p..."
3,The Adventures of Sherlock Holmes,Arthur Conan Doyle,https://www.gutenberg.org/files/1661/1661-h/16...,“It must be recovered.” “We have tried and fai...
4,"Moby Dick; Or, The Whale",Herman Melville,https://www.gutenberg.org/files/2701/2701-h/27...,"The pale Usher—threadbare in coat, heart, body..."
...,...,...,...,...
95,The Elements of Style,William Strunk,https://www.gutenberg.org/files/37134/37134-h/...,It is nearly half past five. We cannot reach t...
96,"Songs of Innocence, and Songs of Experience",William Blake,https://www.gutenberg.org/files/1934/1934-h/19...,58 The Human Abstract 59 Infant Sorrow 61 A Po...
97,Pygmalion,Bernard Shaw,https://www.gutenberg.org/files/3825/3825-h/38...,Titterings. Popular interest in the note taker...
98,Candide,Voltaire,https://www.gutenberg.org/files/19942/19942-h/...,"""What can be the sufficient reason of this phe..."


In [67]:
excerpt_list = gutenberg_df['excerpt'].tolist()
#excerpt_list = [excerpt.replace('\n', ' ') for excerpt in excerpt_list]

In [68]:
MAX_LENGTH = 256
tokenized_excerpts = tokenizer(excerpt_list, padding="max_length", max_length=256, truncation=True, return_tensors="pt")
tokenized_excerpts

{'input_ids': tensor([[   0, 1779, 7343,  ...,    1,    1,    1],
        [   0,   17,   48,  ...,    1,    1,    1],
        [   0, 1779,  127,  ...,    1,    1,    1],
        ...,
        [   0,  565, 7915,  ...,    1,    1,    1],
        [   0,  113, 2264,  ...,    1,    1,    1],
        [   0,  713,   21,  ...,    1,    1,    1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [69]:
preds_list = []
for i in range(len(excerpt_list)):
    token_seq = tokenizer(excerpt_list[i], padding = "max_length", max_length=256, truncation=True, return_tensors="pt")
    preds = model(**token_seq).logits[0].item()
    preds_list.append(preds)

In [71]:
preds_list

[-0.15752682089805603,
 0.4902236759662628,
 -0.7598305940628052,
 0.01562890037894249,
 -2.137132406234741,
 -0.2922174632549286,
 0.2898958921432495,
 -2.342776298522949,
 -0.11357664316892624,
 -0.05659713223576546,
 -0.15762488543987274,
 -3.169496536254883,
 0.553392767906189,
 -1.67536461353302,
 0.4439523220062256,
 -0.9149787425994873,
 -2.443272829055786,
 -0.8899567127227783,
 -1.6969496011734009,
 -2.2286739349365234,
 -0.08285431563854218,
 -0.664053738117218,
 -0.2971412241458893,
 -0.6967737674713135,
 -0.04978809133172035,
 -0.17781071364879608,
 -0.7128227353096008,
 0.5432456731796265,
 -2.866455316543579,
 -1.332999348640442,
 -2.3190839290618896,
 -2.1002449989318848,
 -0.19599579274654388,
 -0.32079029083251953,
 -0.20038233697414398,
 0.09612865000963211,
 -2.0747807025909424,
 -0.8797600269317627,
 -0.2424597293138504,
 -0.9138988256454468,
 -0.9937719106674194,
 -2.566276788711548,
 0.06945429742336273,
 -0.13268306851387024,
 -2.410763740539551,
 -0.198832005262

In [72]:
gutenberg_preds = pd.DataFrame({'book': gutenberg_df['book'],
                                'author': gutenberg_df['author'],
                                'url': gutenberg_df['url'],
                                'excerpt': gutenberg_df['excerpt'],
                                'target': preds_list})

In [56]:
gutenberg_preds.to_csv('./Gutenberg-Predictions/gutenberg-predictions.csv', index = False, encoding = 'utf-8-sig')